In [30]:
import pandas as pd
import numpy as np
import re
import unicodedata

# ========= 0) CONFIG =========
PATH_INPUT = "/Users/victorbonilla/Documents/MIAD/3er Semestre/1er ciclo/StoryTelling/Semana 3/Ask A Manager Salary Survey 2021.xlsx"
SHEET_NAME = 0  # <- MUY IMPORTANTE: 0 = primera hoja (evita dict)

# TRM del día (USD -> COP) del ejercicio (créditos del dashboard)
TRM_USD_COP = 3670.20
TRM_DATE = "2026-02-08"
TRM_SOURCE = "TRM del día (manual / fuente indicada en el informe)"

# Fuente de FX (para otras monedas vs COP) — puedes citarla como referencia
FX_SOURCE = "Tabla FX parametrizada (manual)"

# Tasas: 1 unidad de moneda = COP  (AJUSTA según tu fecha)
# Si tu consigna exige “del día”, llena estos valores con tu evidencia.
rates_to_cop = {
    "COP": 1.0,
    "USD": TRM_USD_COP,
    "EUR": 4250.00,
    "GBP": 5000.00,
    "CAD": 2900.00,
    "AUD": 2600.00,
    "NZD": 2400.00,
    "CHF": 4500.00,
    "SEK": 380.00,
    "JPY": 26.00,
    "ZAR": 210.00,
    "HKD": 500.00
}

# Umbrales defensivos (igual que tu notebook)
SALARY_MIN = 1000
SALARY_MAX = 1_000_000
BONUS_MAX  = 500_000

print("✅ Config OK")
print("TRM_USD_COP:", TRM_USD_COP, "| TRM_DATE:", TRM_DATE)


✅ Config OK
TRM_USD_COP: 3670.2 | TRM_DATE: 2026-02-08


In [32]:
df = pd.read_excel(PATH_INPUT, sheet_name=SHEET_NAME)

print("✅ Cargado:", PATH_INPUT)
print("Filas:", df.shape[0], "| Columnas:", df.shape[1])
print("\nColumnas:")
print(df.columns.tolist())

summary = pd.DataFrame({
    "tipo": df.dtypes.astype(str),
    "nulos": df.isna().sum(),
    "pct_nulos": (df.isna().mean() * 100).round(2),
    "unicos": df.nunique(dropna=True)
}).sort_values("pct_nulos", ascending=False)

print("\n--- Resumen (tipos, nulos, únicos) ---")
display(summary.head(20))

summary.to_csv("eda_resumen_variables.csv", index=True)
print("✅ Generado: eda_resumen_variables.csv")


✅ Cargado: /Users/victorbonilla/Documents/MIAD/3er Semestre/1er ciclo/StoryTelling/Semana 3/Ask A Manager Salary Survey 2021.xlsx
Filas: 28190 | Columnas: 18

Columnas:
['Timestamp', 'How old are you?', 'What industry do you work in?', 'Job title', 'If your job title needs additional context, please clarify here:', "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)", 'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.', 'Please indicate the currency', 'If "Other," please indicate the currency here: ', 'If your income needs additional context, please provide it here:', 'What country do you work in?', "If you're in the U.S., what state do you work in?", 'What city do you wor

,tipo,nulos,pct_nulos,unicos
"If ""Other,"" please indicate the currency here:",object,27971,99.22,129
"If your income needs additional context, please provide it here:",object,25139,89.18,2987
"If your job title needs additional context, please clarify here:",object,20901,74.14,7026
"How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",float64,7356,26.09,849
"If you're in the U.S., what state do you work in?",object,5062,17.96,137
What is your highest level of education completed?,object,236,0.84,6
What is your race? (Choose all that apply.),object,190,0.67,52
What is your gender?,object,182,0.65,5
What city do you work in?,object,82,0.29,4868
What industry do you work in?,object,81,0.29,1225


✅ Generado: eda_resumen_variables.csv


In [34]:
cols = {c.lower().strip(): c for c in df.columns}

def pick_col(*candidates):
    for k in candidates:
        k = k.lower().strip()
        if k in cols:
            return cols[k]
    return None

COL_TIMESTAMP = pick_col("timestamp")
COL_INDUSTRY  = pick_col("what industry do you work in?")
COL_JOB       = pick_col("job title")
COL_AGE       = pick_col("how old are you?")
COL_COUNTRY   = pick_col("what country do you work in?")
COL_STATE     = pick_col("if you're in the u.s., what state do you work in?")
COL_CITY      = pick_col("what city do you work in?")

COL_SALARY = pick_col(
    "what is your annual salary? (you'll indicate the currency in a later question. if you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)",
    "what is your annual salary?"
)
COL_BONUS = pick_col(
    "how much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? please only include monetary compensation here, not the value of benefits.",
    "how much additional monetary compensation do you get, if any?"
)
COL_CURR   = pick_col("please indicate the currency", "currency")
COL_CURR_O = pick_col('if "other," please indicate the currency here:', 'if "other," please indicate the currency here: ')

# Opcionales (módulo 4)
COL_EXP_ALL   = pick_col("how many years of professional work experience do you have overall?")
COL_EDU       = pick_col("what is your highest level of education completed?")
COL_GENDER    = pick_col("what is your gender?")
COL_RACE      = pick_col("what is your race? (choose all that apply.)")

print("Country:", COL_COUNTRY)
print("City:", COL_CITY)
print("Salary:", COL_SALARY)
print("Bonus:", COL_BONUS)
print("Currency:", COL_CURR)

needed = [COL_COUNTRY, COL_CITY, COL_INDUSTRY, COL_SALARY, COL_BONUS, COL_CURR]
missing = [x for x in needed if x is None]
if missing:
    raise ValueError(f"Faltan columnas clave. Revisa nombres. Missing: {missing}")


Country: What country do you work in?
City: What city do you work in?
Salary: What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)
Bonus: How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.
Currency: Please indicate the currency


In [36]:
def norm_text(x):
    if pd.isna(x): return np.nan
    return str(x).strip()

def norm_key(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = s.replace(".", "")
    return s

def parse_money(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    s = s.replace(",", "")
    s = re.sub(r"[$£€]", "", s)
    s = s.replace(" ", "")
    if s == "":
        return np.nan
    return pd.to_numeric(s, errors="coerce")

def normalize_text_basic(s):
    if pd.isna(s): return np.nan
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("utf-8")
    s = s.strip().lower()
    return s

def clean_city(s):
    if pd.isna(s): return np.nan
    s = normalize_text_basic(s)
    s = re.sub(r"\(.*?\)", "", s)           # quita paréntesis
    s = s.split(",")[0].strip()             # "Boston, MA" -> "Boston"
    non_city = {"remote","prefer not to answer","n/a","na","none",""}
    if s in non_city:
        return np.nan
    city_map = {
        "nyc":"new york city",
        "new york":"new york city",
        "washington dc":"washington",
        "dc":"washington",
        "san fran":"san francisco",
        "sf":"san francisco",
        "st louis":"saint louis",
        "st. louis":"saint louis",
    }
    s = city_map.get(s, s)
    return s.title()

def build_city_context(row):
    city = row["city_clean"]
    state = row.get(COL_STATE, np.nan)
    if pd.isna(city): return np.nan
    if pd.notna(state) and str(state).strip() != "":
        return f"{city}, {str(state).strip()}"
    return city

def clean_currency(row):
    cur = str(row[COL_CURR]).strip() if pd.notna(row[COL_CURR]) else ""
    oth = ""
    if COL_CURR_O is not None and COL_CURR_O in row and pd.notna(row[COL_CURR_O]):
        oth = str(row[COL_CURR_O]).strip()

    if cur != "Other":
        return cur if cur != "" else np.nan

    m = re.search(r"\b[A-Z]{3}\b", oth.upper())
    if m:
        return m.group(0)
    return np.nan


In [38]:
# ----- COUNTRY -----
df["country_raw"] = df[COL_COUNTRY].apply(norm_text)
df["country_key"] = df[COL_COUNTRY].apply(norm_key)

country_map = {
    "us":"United States","usa":"United States","u s":"United States","u s a":"United States",
    "united states":"United States","united states of america":"United States","united states america":"United States",
    "uk":"United Kingdom","united kingdom":"United Kingdom","england":"United Kingdom","scotland":"United Kingdom",
    "wales":"United Kingdom","northern ireland":"United Kingdom",
}

df["country_clean"] = df["country_key"].map(country_map)
df.loc[df["country_clean"].isna(), "country_clean"] = (
    df.loc[df["country_clean"].isna(), "country_raw"].astype(str).str.strip()
)

print("✅ Country clean top:")
print(df["country_clean"].value_counts().head(15))

# ----- CITY -----
df["city_clean"] = df[COL_CITY].apply(clean_city)
df["city_context"] = df.apply(build_city_context, axis=1)

print("✅ City clean listo.")
print("Ciudades únicas antes:", df[COL_CITY].nunique(dropna=True))
print("Ciudades únicas después:", df["city_clean"].nunique(dropna=True))


✅ Country clean top:
country_clean
United States      23111
Canada              1650
United Kingdom      1564
Australia            387
Germany              193
Ireland              122
New Zealand          117
France                67
Netherlands           54
Spain                 47
Sweden                41
Switzerland           36
Belgium               35
Japan                 28
The Netherlands       28
Name: count, dtype: int64
✅ City clean listo.
Ciudades únicas antes: 4868
Ciudades únicas después: 3439


In [40]:
df["currency_clean"] = df.apply(clean_currency, axis=1)

df["salary_num"] = df[COL_SALARY].apply(parse_money)
df["bonus_num"]  = df[COL_BONUS].apply(parse_money)

# regla del notebook: si salario < 1000, asume "miles"
df["salary_clean"] = df["salary_num"].copy()
df.loc[df["salary_clean"].notna() & (df["salary_clean"] < 1000), "salary_clean"] *= 1000

df["salary_valid"] = df["salary_clean"].between(SALARY_MIN, SALARY_MAX, inclusive="both")
df["bonus_valid"]  = df["bonus_num"].isna() | df["bonus_num"].between(0, BONUS_MAX, inclusive="both")

print("✅ Salary valid %:", round(df["salary_valid"].mean()*100, 2), "%")
print("✅ Bonus valid  %:", round(df["bonus_valid"].mean()*100, 2), "%")

# Base limpia lista para conversión monetaria
df_model = df[df["salary_valid"] & df["bonus_valid"] & df["currency_clean"].notna()].copy()

print("Registros originales:", len(df))
print("Registros listos (monetario):", len(df_model))

df_model.to_csv("ask_a_manager_model_base.csv", index=False)
print("✅ Generado: ask_a_manager_model_base.csv")


✅ Salary valid %: 99.62 %
✅ Bonus valid  %: 99.88 %
Registros originales: 28190
Registros listos (monetario): 28030
✅ Generado: ask_a_manager_model_base.csv


In [42]:
rename_for_dash = {
    "country_clean": "country",
    "city_clean": "city",
    "city_context": "city_context",
    "currency_clean": "currency",
    "salary_clean": "salary_annual",
    "bonus_num": "bonus_annual",
    COL_INDUSTRY: "industry"
}

# opcionales útiles
if COL_JOB:    rename_for_dash[COL_JOB] = "job_title"
if COL_AGE:    rename_for_dash[COL_AGE] = "age_range"
if COL_STATE:  rename_for_dash[COL_STATE] = "state_us"
if COL_EXP_ALL:rename_for_dash[COL_EXP_ALL] = "experience_overall"
if COL_EDU:    rename_for_dash[COL_EDU] = "education"
if COL_GENDER: rename_for_dash[COL_GENDER] = "gender"
if COL_RACE:   rename_for_dash[COL_RACE] = "race"
if COL_TIMESTAMP: rename_for_dash[COL_TIMESTAMP] = "timestamp"

df_dash = df_model.rename(columns=rename_for_dash).copy()

cols_keep = [
    "timestamp",
    "country","state_us","city","city_context",
    "industry",
    "currency","salary_annual","bonus_annual",
    "experience_overall","education","gender","race","job_title","age_range"
]
cols_keep = [c for c in cols_keep if c in df_dash.columns]
df_dash = df_dash[cols_keep].copy()

print("✅ Dataset clean (pre-COP) listo:", df_dash.shape)
df_dash.head(5)


✅ Dataset clean (pre-COP) listo: (28030, 15)


,timestamp,country,state_us,city,city_context,industry,currency,salary_annual,bonus_annual,experience_overall,education,gender,race,job_title,age_range
0,2021-04-27 11:02:09.743,United States,Massachusetts,Boston,"Boston, Massachusetts",Education (Higher Education),USD,55000,0.0,5-7 years,Master's degree,Woman,White,Research and Instruction Librarian,25-34
1,2021-04-27 11:02:21.562,United Kingdom,NaN,Cambridge,Cambridge,Computing or Tech,GBP,54600,4000.0,8 - 10 years,College degree,Non-binary,White,Change & Internal Communications Manager,25-34
2,2021-04-27 11:02:38.125,United States,Tennessee,Chattanooga,"Chattanooga, Tennessee","Accounting, Banking & Finance",USD,34000,NaN,2 - 4 years,College degree,Woman,White,Marketing Specialist,25-34
3,2021-04-27 11:02:40.643,United States,Wisconsin,Milwaukee,"Milwaukee, Wisconsin",Nonprofits,USD,62000,3000.0,8 - 10 years,College degree,Woman,White,Program Manager,25-34
4,2021-04-27 11:02:41.793,United States,South Carolina,Greenville,"Greenville, South Carolina","Accounting, Banking & Finance",USD,60000,7000.0,8 - 10 years,College degree,Woman,White,Accounting Manager,25-34


In [44]:
def fx_to_cop(currency):
    if pd.isna(currency):
        return np.nan
    return rates_to_cop.get(str(currency).upper().strip(), np.nan)

df_dash["fx_to_cop_aplicada"] = df_dash["currency"].apply(fx_to_cop)

# Créditos / trazabilidad (para dashboard)
df_dash["trm_usd_cop"] = TRM_USD_COP
df_dash["trm_date"] = TRM_DATE
df_dash["trm_source"] = TRM_SOURCE
df_dash["fx_source"] = FX_SOURCE

# Conversiones COP (por persona)
df_dash["salario_anual_cop"] = df_dash["salary_annual"] * df_dash["fx_to_cop_aplicada"]
df_dash["compensaciones_cop"] = df_dash["bonus_annual"].fillna(0) * df_dash["fx_to_cop_aplicada"]
df_dash["total_compensacion_cop"] = df_dash["salario_anual_cop"].fillna(0) + df_dash["compensaciones_cop"].fillna(0)

print("✅ % filas sin tasa aplicada:", round(df_dash["fx_to_cop_aplicada"].isna().mean()*100,2), "%")
print("Monedas sin tasa (top):")
print(df_dash.loc[df_dash["fx_to_cop_aplicada"].isna(), "currency"].value_counts().head(15))

df_dash[["currency","fx_to_cop_aplicada","salary_annual","salario_anual_cop","trm_date"]].head(10)


✅ % filas sin tasa aplicada: 2.12 %
Monedas sin tasa (top):
currency
AUD/NZD    505
SGD         13
DKK         10
MYR          8
NOK          8
PLN          7
BRL          7
INR          6
ILS          5
PHP          3
CZK          3
TRY          2
CNY          2
RMB          2
THB          2
Name: count, dtype: int64


,currency,fx_to_cop_aplicada,salary_annual,salario_anual_cop,trm_date
0,USD,3670.2,55000,201861000.0,2026-02-08
1,GBP,5000.0,54600,273000000.0,2026-02-08
2,USD,3670.2,34000,124786800.0,2026-02-08
3,USD,3670.2,62000,227552400.0,2026-02-08
4,USD,3670.2,60000,220212000.0,2026-02-08
5,USD,3670.2,62000,227552400.0,2026-02-08
6,USD,3670.2,33000,121116600.0,2026-02-08
7,USD,3670.2,50000,183510000.0,2026-02-08
8,USD,3670.2,112000,411062400.0,2026-02-08
9,USD,3670.2,45000,165159000.0,2026-02-08


In [46]:
# Columnas mínimas para cumplir módulos + créditos
cols_final = [
    "country","city","city_context",
    "industry",
    "salario_anual_cop","compensaciones_cop","total_compensacion_cop",
    "trm_usd_cop","trm_date","trm_source",
    "fx_to_cop_aplicada","fx_source"
]

# módulo 4 (libre): deja al menos una dimensión adicional si existe
for extra in ["experience_overall","education","gender","job_title","age_range","state_us"]:
    if extra in df_dash.columns:
        cols_final.append(extra)

df_final = df_dash[cols_final].copy()

df_final.to_csv("ask_a_manager_looker_FINAL_COP_TRM.csv", index=False, encoding="utf-8-sig")
df_final.to_excel("ask_a_manager_looker_FINAL_COP_TRM.xlsx", index=False)

print("✅ Dataset final generado:")
print(" - ask_a_manager_looker_FINAL_COP_TRM.csv")
print(" - ask_a_manager_looker_FINAL_COP_TRM.xlsx")
df_final.head(5)


✅ Dataset final generado:
 - ask_a_manager_looker_FINAL_COP_TRM.csv
 - ask_a_manager_looker_FINAL_COP_TRM.xlsx


,country,city,city_context,industry,salario_anual_cop,compensaciones_cop,total_compensacion_cop,trm_usd_cop,trm_date,trm_source,fx_to_cop_aplicada,fx_source,experience_overall,education,gender,job_title,age_range,state_us
0,United States,Boston,"Boston, Massachusetts",Education (Higher Education),201861000.0,0.0,201861000.0,3670.2,2026-02-08,TRM del día (manual / fuente indicada en el in...,3670.2,Tabla FX parametrizada (manual),5-7 years,Master's degree,Woman,Research and Instruction Librarian,25-34,Massachusetts
1,United Kingdom,Cambridge,Cambridge,Computing or Tech,273000000.0,20000000.0,293000000.0,3670.2,2026-02-08,TRM del día (manual / fuente indicada en el in...,5000.0,Tabla FX parametrizada (manual),8 - 10 years,College degree,Non-binary,Change & Internal Communications Manager,25-34,NaN
2,United States,Chattanooga,"Chattanooga, Tennessee","Accounting, Banking & Finance",124786800.0,0.0,124786800.0,3670.2,2026-02-08,TRM del día (manual / fuente indicada en el in...,3670.2,Tabla FX parametrizada (manual),2 - 4 years,College degree,Woman,Marketing Specialist,25-34,Tennessee
3,United States,Milwaukee,"Milwaukee, Wisconsin",Nonprofits,227552400.0,11010600.0,238563000.0,3670.2,2026-02-08,TRM del día (manual / fuente indicada en el in...,3670.2,Tabla FX parametrizada (manual),8 - 10 years,College degree,Woman,Program Manager,25-34,Wisconsin
4,United States,Greenville,"Greenville, South Carolina","Accounting, Banking & Finance",220212000.0,25691400.0,245903400.0,3670.2,2026-02-08,TRM del día (manual / fuente indicada en el in...,3670.2,Tabla FX parametrizada (manual),8 - 10 years,College degree,Woman,Accounting Manager,25-34,South Carolina


In [48]:
import pandas as pd

file_name = "BD_Modelada_Dashboard_TRM.xlsx"

with pd.ExcelWriter(file_name, engine="xlsxwriter") as writer:

    df_final.to_excel(writer, 
                      sheet_name="BD_Modelada", 
                      index=False)

    workbook  = writer.book
    worksheet = writer.sheets["BD_Modelada"]

    # ===== Formato encabezados =====
    header_format = workbook.add_format({
        "bold": True,
        "text_wrap": False,
        "valign": "middle",
        "align": "center",
        "border": 1
    })

    for col_num, value in enumerate(df_final.columns):
        worksheet.write(0, col_num, value, header_format)

    # ===== Formato moneda =====
    currency_format = workbook.add_format({'num_format': '$#,##0'})

    money_cols = [
        "salario_anual_cop",
        "compensaciones_cop",
        "total_compensacion_cop"
    ]

    for col in money_cols:
        if col in df_final.columns:
            idx = df_final.columns.get_loc(col)
            worksheet.set_column(idx, idx, 18, currency_format)

    # ===== Ajustar ancho automático =====
    for i, col in enumerate(df_final.columns):
        width = max(
            df_final[col].astype(str).map(len).max(),
            len(col)
        ) + 2
        
        width = min(width, 35)
        worksheet.set_column(i, i, width)

    # ===== Congelar encabezados =====
    worksheet.freeze_panes(1, 0)

    # ===== Convertir en tabla (MUY PRO) =====
    rows, cols = df_final.shape

    worksheet.add_table(
        0, 0, rows, cols-1,
        {
            "columns": [{"header": col} for col in df_final.columns],
            "style": "Table Style Medium 2"
        }
    )

print("✅ EXCEL GENERADO CON ÉXITO")
print("Archivo:", file_name)


✅ EXCEL GENERADO CON ÉXITO
Archivo: BD_Modelada_Dashboard_TRM.xlsx
